In [1]:
import pandas as pd

raw = pd.read_excel("C:/Users/Lenovo/Downloads/tan/Assessment Data Asset Dummy.xlsx")
raw

,Funcloc,Alamat1,Alamat2,Alamat3,Alamat4,SiteName
0,100000000002,Desa Alun Dua,Desa Alun Dua,Kecamatan Kerkap,BENGKULU UTARA,Steele-Phillips
1,100000000003,Desa Padang Maninjau,Kel. Desa Padang Maninjau,Kec. Aek Kuo,LABUHANBATU UTARA,"Herrera, Barnes and Taylor"
2,100000000004,Desa Bio-Bio,Kel. Bandar Durian,Kec. Aek Natas,LABUHANBATU UTARA,Lee Group
3,100000000005,Rintis Kacang,Desa Pangakalan,Kec. Aek Natas,LABUHANBATU UTARA,Barton Inc
4,100000000006,Dusun VI Sidodadi,Desa Marbau Selatan,Kec. Marbau,LABUHANBATU UTARA,"Cole, Molina and Williams"
...,...,...,...,...,...,...
4903,100000059183,Dusun Tiga Desa Aek Nauli 2 Kec.Pollung,Ds Aek Nauli 2,Pollung,Humbang Hasundutan,Hurley Inc
4904,100000059265,Desa Pansur Natolu Kec.Pangaribuan,Ds Pansur Natolu,Pangaribuan,Tapanuli Utara,Wright-Garza
4905,100000059267,Jl.Banglas Gg Damai RT.02/02,Ds Selatpanjang Timur,Tebing Tinggi,Kepulauan Meranti,Benton-Perez
4906,100000059268,Desa Banuaji I Kec.Adiankoting,Ds Banuaji I,Adiankoting,Tapanuli Utara,Ingram-Brown


In [2]:
master = pd.read_excel("C:/Users/Lenovo/Downloads/tan/City Indonesia.xlsx")
master

,City,CityCode,Province,Region,RegionalCode
0,Kabupaten Aceh Barat,MBO,Provinsi Aceh,Sumbagut,1
1,Kabupaten Aceh Barat Daya,BPD,Provinsi Aceh,Sumbagut,1
2,Kabupaten Aceh Besar,JTH,Provinsi Aceh,Sumbagut,1
3,Kabupaten Aceh Jaya,CAG,Provinsi Aceh,Sumbagut,1
4,Kabupaten Aceh Selatan,TTN,Provinsi Aceh,Sumbagut,1
...,...,...,...,...,...
509,Kota Ternate,TTE,Provinsi Maluku Utara,Sulawesi,10
510,Kota Tidore Kepulauan,TDR,Provinsi Maluku Utara,Sulawesi,10
511,Kota Tomohon,TMH,Provinsi Sulawesi Utara,Sulawesi,10
512,Kota Tual,TUL,Provinsi Maluku,Puma,11


### 1. Data Validation

In [3]:
# a. Check for required fields
# b. Validate data formats (province(provinsi), district(kabkot), etc)
# c. Verify numeric values are within acceptable ranges

master.info()
raw.info()

master.isnull().sum() #tidak ada
raw.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   City          514 non-null    object
 1   CityCode      514 non-null    object
 2   Province      514 non-null    object
 3   Region        514 non-null    object
 4   RegionalCode  514 non-null    int64 
dtypes: int64(1), object(4)
memory usage: 20.2+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4908 entries, 0 to 4907
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Funcloc   4908 non-null   int64 
 1   Alamat1   4908 non-null   object
 2   Alamat2   4884 non-null   object
 3   Alamat3   4877 non-null   object
 4   Alamat4   4908 non-null   object
 5   SiteName  4908 non-null   object
dtypes: int64(1), object(5)
memory usage: 230.2+ KB


Funcloc      0
Alamat1      0
Alamat2     24
Alamat3     31
Alamat4      0
SiteName     0
dtype: int64

In [4]:
# d. Validate referential integrity (between asset data and master data)
raw['is_valid'] = raw['Alamat4'].isin(master['City'])
invalid_data = raw[~raw['is_valid']]
print("Jumlah data invalid:", len(invalid_data))

Jumlah data invalid: 4908


### 2. Data Cleansing

In [5]:
# a. Standardize date formats -> standarisasi alamat4 untuk case
raw['Alamat4']   = raw['Alamat4'].str.title().str.strip()
raw['Alamat4'] 

0           Bengkulu Utara
1        Labuhanbatu Utara
2        Labuhanbatu Utara
3        Labuhanbatu Utara
4        Labuhanbatu Utara
               ...        
4903    Humbang Hasundutan
4904        Tapanuli Utara
4905     Kepulauan Meranti
4906        Tapanuli Utara
4907                 Dairi
Name: Alamat4, Length: 4908, dtype: object

In [6]:
# b. Normalize payment method values -> normalisasi alamat 4
freq = raw["Alamat4"].value_counts()
raw["Alamat4Freq"] = raw["Alamat4"].map(freq)

raw[["Alamat4", "Alamat4Freq"]]

,Alamat4,Alamat4Freq
0,Bengkulu Utara,15
1,Labuhanbatu Utara,19
2,Labuhanbatu Utara,19
3,Labuhanbatu Utara,19
4,Labuhanbatu Utara,19
...,...,...
4903,Humbang Hasundutan,14
4904,Tapanuli Utara,16
4905,Kepulauan Meranti,10
4906,Tapanuli Utara,16


In [7]:
# c. Clean and standardize phone numbers
from rapidfuzz import process, fuzz
import pandas as pd

city_list = master["City"].tolist()

def match_city_full(x):
    if pd.isna(x) or x.strip() == "":
        return None
    result = process.extractOne(x, city_list, scorer=fuzz.token_sort_ratio)
    if result is None:
        return None
    match, score, _ = result
    return match if score >= 50 else None  # threshold 

# Terapkan ke raw
raw["Alamat4"] = raw["Alamat4"].apply(match_city_full)


In [8]:
# d. Remove or handle invalid values
raw["Alamat2"].fillna("Unknown", inplace=True)
raw["Alamat3"].fillna("Unknown", inplace=True)

In [9]:
#e. Use column “alamat4” for Kabupaten/Kota data. However the data did not havethe same format as the master data (“City Indonesia.xlsx”). Thus, you have to transform data within that column, align with the master data.

def transform_alamat(x):
    x = str(x).strip().title()  # Bersihkan dan title case
    x_clean = x.replace("Kota ", "").replace(", Kota", "").replace("Kabupaten ", "").replace(", Kabupaten", "").strip()
    
    # Tentukan prefix
    if "Kota" in x or ", Kota" in x:
        prefix = "Kota"
    elif "Kabupaten" in x or ", Kabupaten" in x:
        prefix = "Kabupaten"
    else:
        prefix = "Kabupaten"  # default jika tidak ada kata Kota/Kabupaten
    
    return f"{prefix} {x_clean}"

raw["Alamat4"] = raw["Alamat4"].apply(transform_alamat)

raw["Alamat4"] 

0                      Kota Bengkulu
1        Kabupaten Labuhanbatu Utara
2        Kabupaten Labuhanbatu Utara
3        Kabupaten Labuhanbatu Utara
4        Kabupaten Labuhanbatu Utara
                    ...             
4903    Kabupaten Humbang Hasundutan
4904        Kabupaten Tapanuli Utara
4905     Kabupaten Kepulauan Meranti
4906        Kabupaten Tapanuli Utara
4907                 Kabupaten Dairi
Name: Alamat4, Length: 4908, dtype: object

In [10]:
raw['is_valid'] = raw['Alamat4'].isin(master['City'])
invalid_data = raw[~raw['is_valid']]
print("Jumlah data invalid:", len(invalid_data))

Jumlah data invalid: 263


### 3. Data Transformation 

In [11]:
# a. Convert funcloc to proper data format 
raw["Funcloc"] = pd.to_numeric(raw["Funcloc"], errors="coerce")


In [12]:
# b. Indicate order
raw = raw.sort_values(by=["Funcloc"]).reset_index(drop=True)
raw

,Funcloc,Alamat1,Alamat2,Alamat3,Alamat4,SiteName,is_valid,Alamat4Freq
0,100000000002,Desa Alun Dua,Desa Alun Dua,Kecamatan Kerkap,Kota Bengkulu,Steele-Phillips,True,15
1,100000000003,Desa Padang Maninjau,Kel. Desa Padang Maninjau,Kec. Aek Kuo,Kabupaten Labuhanbatu Utara,"Herrera, Barnes and Taylor",True,19
2,100000000004,Desa Bio-Bio,Kel. Bandar Durian,Kec. Aek Natas,Kabupaten Labuhanbatu Utara,Lee Group,True,19
3,100000000005,Rintis Kacang,Desa Pangakalan,Kec. Aek Natas,Kabupaten Labuhanbatu Utara,Barton Inc,True,19
4,100000000006,Dusun VI Sidodadi,Desa Marbau Selatan,Kec. Marbau,Kabupaten Labuhanbatu Utara,"Cole, Molina and Williams",True,19
...,...,...,...,...,...,...,...,...
4903,100000059183,Dusun Tiga Desa Aek Nauli 2 Kec.Pollung,Ds Aek Nauli 2,Pollung,Kabupaten Humbang Hasundutan,Hurley Inc,True,14
4904,100000059265,Desa Pansur Natolu Kec.Pangaribuan,Ds Pansur Natolu,Pangaribuan,Kabupaten Tapanuli Utara,Wright-Garza,True,16
4905,100000059267,Jl.Banglas Gg Damai RT.02/02,Ds Selatpanjang Timur,Tebing Tinggi,Kabupaten Kepulauan Meranti,Benton-Perez,True,10
4906,100000059268,Desa Banuaji I Kec.Adiankoting,Ds Banuaji I,Adiankoting,Kabupaten Tapanuli Utara,Ingram-Brown,True,16


In [13]:
#b. & c. Create column Internal Site ID
 
raw = raw.merge(
    master[["City", "CityCode", "RegionalCode"]],
    left_on="Alamat4",
    right_on="City",
    how="left",
    indicator=True
)

# Pastikan tipe data
raw["RegionalCode"] = raw["RegionalCode"].fillna(0).astype(int)
raw["CityCode"] = raw["CityCode"].astype(str)


# Generate running number CCC per city
raw["CCC"] = raw.groupby("CityCode").cumcount() + 1
raw["CCC"] = raw["CCC"].astype(str).str.zfill(3)  # 001, 002, ...

# Format RegionalCode 2 digit
raw["RegionalCode2"] = raw["RegionalCode"].astype(str).str.zfill(2)

# Buat Internal Site ID = AAA-BB-CCC
raw["Internal Site ID"] = raw["CityCode"] + "-" + raw["RegionalCode2"] + "-" + raw["CCC"]

# -------- Reorder kolom: Internal Site ID di paling kiri, Funcloc di paling kanan --------
cols = ["Internal Site ID"] + [c for c in raw.columns if c not in ["Internal Site ID", "Funcloc"]] + ["Funcloc"]
raw = raw[cols]



### 4. Error Handling 

In [14]:
#a. Log invalid records 
invalid_records = raw[raw["_merge"] != "both"]
if not invalid_records.empty:
    invalid_records.to_excel("InvalidLog.xlsx", index=False)
    print(f"{len(invalid_records)} record invalid dicatat di 'InvalidLog.xlsx'")

# Simpan hasil ke Excel
raw.to_excel("DataRawFuncloc1.xlsx", index=False)
print("Hasil sudah disimpan di 'DataRawFuncloc1.xlsx'")

263 record invalid dicatat di 'InvalidLog.xlsx'
Hasil sudah disimpan di 'DataRawFuncloc1.xlsx'


In [15]:
raw

,Internal Site ID,Alamat1,Alamat2,Alamat3,Alamat4,SiteName,is_valid,Alamat4Freq,City,CityCode,RegionalCode,_merge,CCC,RegionalCode2,Funcloc
0,BGL-03-001,Desa Alun Dua,Desa Alun Dua,Kecamatan Kerkap,Kota Bengkulu,Steele-Phillips,True,15,Kota Bengkulu,BGL,3,both,001,03,100000000002
1,AKK-01-001,Desa Padang Maninjau,Kel. Desa Padang Maninjau,Kec. Aek Kuo,Kabupaten Labuhanbatu Utara,"Herrera, Barnes and Taylor",True,19,Kabupaten Labuhanbatu Utara,AKK,1,both,001,01,100000000003
2,AKK-01-002,Desa Bio-Bio,Kel. Bandar Durian,Kec. Aek Natas,Kabupaten Labuhanbatu Utara,Lee Group,True,19,Kabupaten Labuhanbatu Utara,AKK,1,both,002,01,100000000004
3,AKK-01-003,Rintis Kacang,Desa Pangakalan,Kec. Aek Natas,Kabupaten Labuhanbatu Utara,Barton Inc,True,19,Kabupaten Labuhanbatu Utara,AKK,1,both,003,01,100000000005
4,AKK-01-004,Dusun VI Sidodadi,Desa Marbau Selatan,Kec. Marbau,Kabupaten Labuhanbatu Utara,"Cole, Molina and Williams",True,19,Kabupaten Labuhanbatu Utara,AKK,1,both,004,01,100000000006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4903,DLS-01-014,Dusun Tiga Desa Aek Nauli 2 Kec.Pollung,Ds Aek Nauli 2,Pollung,Kabupaten Humbang Hasundutan,Hurley Inc,True,14,Kabupaten Humbang Hasundutan,DLS,1,both,014,01,100000059183
4904,TRT-01-015,Desa Pansur Natolu Kec.Pangaribuan,Ds Pansur Natolu,Pangaribuan,Kabupaten Tapanuli Utara,Wright-Garza,True,16,Kabupaten Tapanuli Utara,TRT,1,both,015,01,100000059265
4905,TTG-02-010,Jl.Banglas Gg Damai RT.02/02,Ds Selatpanjang Timur,Tebing Tinggi,Kabupaten Kepulauan Meranti,Benton-Perez,True,10,Kabupaten Kepulauan Meranti,TTG,2,both,010,02,100000059267
4906,TRT-01-016,Desa Banuaji I Kec.Adiankoting,Ds Banuaji I,Adiankoting,Kabupaten Tapanuli Utara,Ingram-Brown,True,16,Kabupaten Tapanuli Utara,TRT,1,both,016,01,100000059268


In [16]:
print(raw["Alamat4"].tolist())


['Kota Bengkulu', 'Kabupaten Labuhanbatu Utara', 'Kabupaten Labuhanbatu Utara', 'Kabupaten Labuhanbatu Utara', 'Kabupaten Labuhanbatu Utara', 'Kabupaten Labuhanbatu Utara', 'Kota Ambon', 'Kota Ambon', 'Kota Ambon', 'Kabupaten Maluku Tengah', 'Kabupaten Tojo Una-Una', 'Kabupaten Minahasa Utara', 'Kabupaten Minahasa Utara', 'Kabupaten Minahasa Utara', 'Kabupaten Way Kanan', 'Kabupaten Way Kanan', 'Kabupaten Way Kanan', 'Kabupaten Way Kanan', 'Kabupaten Way Kanan', 'Kabupaten Way Kanan', 'Kabupaten Way Kanan', 'Kota Bandar Lampung', 'Kota Bandar Lampung', 'Kota Bandar Lampung', 'Kota Bandar Lampung', 'Kabupaten Bondowoso', 'Kabupaten Bengkayang', 'Kabupaten Morowali', 'Kota Bengkulu', 'Kota Bima', 'Kota Bima', 'Kota Bitung', 'Kota Banjarbaru', 'Kota Banjarbaru', 'Kabupaten Kampar', 'Kabupaten Kampar', 'Kabupaten Kampar', 'Kabupaten Merangin', 'Kabupaten Merangin', 'Kabupaten Merangin', 'Kabupaten Merangin', 'Kota Bekasi', 'Kota Bekasi', 'Kota Bekasi', 'Kota Bekasi', 'Kota Bekasi', 'Kota B